<a href="https://colab.research.google.com/github/rproner1/MATH4060/blob/main/MidtermProject1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import itertools
import jax.numpy as jnp
import matplotlib.pyplot as plt
from jax import random as jrandom
from jax import nn as jnn
import random
import time
import math

# Problem 1 - Simulating a Markov chain with uniform random variables

Consider a Markov chain on the state space $S = \{0, 1, . . . , N_{state}−1\}$ with an $N_{state} \times N_{state}$ transition matrix
$P$. Write a program that inputs the transition matrix $P$ (as a numpy array of shape $(N_{state},N_{state})$, an initial
state $x_0 \in S$, a random uniform variable $U \in (0, 1)$ and returns a random state $X_1 \in S$ so that $X_1$ is a sample
of the Markov chain at time 1. For full effciency marks, minimize the number of if statements/for loops that are
needed by using numpy vector functions instead. (Hint: Its possible to use make a version of this that uses the
numpy function searchsorted)

In [ ]:
def Markov_chain_sim(P,X_0,U):
  '''Given X_0 and the transition matrix P, gives a possible X_1 
     using a uniform [0,1] random variable'''
  #----------------
  #Input: 
  #  P = An array of shape (N,N) where the entries are non-negative and each row sums to 1  
  #  x_0 = An integer from the state space [0,N-1]
  #  U = A real number between [0,1] which is generated as a uniform random variable
  #Ouput:
  #  An integer from the state space [0,N-1] which gives the location of the Markov chain at time X_1
  # ----------------


  # Number of states
  N_state = P.shape[1]
  
  # State space
  S = list(range(N_state))

  if X_0 >= N_state:
    raise Exception("Error: nonexistent state entered. Please enter a state in {0, 1, ..., N_state - 1}.")

  # initial state vector
  V_0 = jnp.array([1.0 if s == X_0 else 0.0 for s in S])
  print(V_0)

  V_1 = jnp.matmul(V_0, P)
  print("v1 =", V_1)
  print("cumsum v1 =", jnp.cumsum(V_1))
  print("U = %f" %U)
  X_1 = jnp.searchsorted(jnp.cumsum(V_1), U)

  return X_1

In [ ]:
# initial state
X_0 = 2

# transition matrix
P = jnp.array([[0.5, 0., 0., 0., 0.5],
               [0.7, 0., 0., 0., 0.3],
               [0., 0.9, 0.1, 0., 0.],
               [0, 0.75, 0.25, 0., 0.],
               [0., 0.6, 0., 0.4, 0.]])

U = random.uniform(0, 1)

print("X_1 =", Markov_chain_sim(P, X_0, U))

[0. 0. 1. 0. 0.]
v1 = [0.  0.9 0.1 0.  0. ]
cumsum v1 = [0.  0.9 1.  1.  1. ]
U = 0.660071
X_1 = 1


The reason we use a uniform random variable is to choose a random state $X_1$. By creating a cumulative sum vector and using the `jnp.searchsorted()` function,
We get a random index (state) from our vector $V_1$. Using this method, we account for the probabilities of being in certain states, since for a continuous uniform random variable $X$

$$
P(a \leq X \leq b) = \int_a^b \frac{1}{b-a} dx
$$

As an example, consider the weather model from Problem 2. Our initial state vector $\mathbf{v}_0 $ and transition probability matrix $\mathbf{P}$ are

$$
\mathbf{v}_0 = [0, 0, 1, 0, 0];\qquad \mathbf{P} = 
\begin{bmatrix}
  0.5 & 0 & 0 & 0 & 0.5 \\
  0.7 & 0 & 0 & 0 & 0.3 \\
  0 & 0.9 & 0.1 & 0 & 0 \\
  0 & 0.75 & 0.25 & 0 & 0 \\
  0 & 0.6 & 0 & 0.4 & 0
\end{bmatrix}
$$

The the probabilities of the five states are given by the vector $\mathbf{v}_1 = \mathbf{v}_0 \cdot \mathbf{P} = [0.0, 0.9, 0.1, 0.0, 0.0]$. Then the probability of being in state 1 (indexed from 0) is 

$$
P(0 \leq X \leq 0.9) = \int_0^{0.9} \frac{1}{1-0} dx = 0.90
$$

and the probability of being in state 2 (indexed from 0) is

$$
P(0.9 \leq X \leq 1.0) = \int_{0.9}^{1.0} \frac{1}{1-0} dx = 0.10
$$

So we see that the probability of a random state $X_1$ is proportional to the interval size in the vector $\mathbf{v}_1$. It is easy to see that states 3 and 4 are impossible.

# Problem 2 - A weather model

On any given day, it is either rainy or sunny. A model for the weather is the
following rules which are applied in order (i.e. rule 1 is applied if possible; if it does not apply, then rule 2 is applied etc.)

1. If it is sunny today and it was sunny yesterday, then the forecast for the
next day is 50% rainy - 50% sunny.
2. If it is sunny today, but it was rainy yesterday, then the forecast for the
next day is 30% rainy - 70% sunny
3. Otherwise, if has been rainy for three days in a row, the forecast for the
next day is 10% rainy - 90% sunny.
4. Otherwise, if has been rainy for two days in a row, the forecast for the
next day is 25% rainy - 75% sunny.
5. Otherwise, if it is rainy today, the forecast for the next day is 40% rainy - 60% sunny.

a) Find a Markov chain to model this problem. Be sure to clearly indicate
the state space you are using is, what the transition matrix is and draw the
corresponding directed graph of the Markov chain.

b) Write a program that inputs an integer n and outputs the exact (up to
computer precision) probability that it is rainy n days from now given that it is sunny today. Draw a graph of this probability as function of $n$ for $n \in [0, 100]$.

c) Calculate the exact value of
$ \lim_{n\to ∞}P(\text{Rainy on day n}|\text{Sunny today and sunny yesterday})$

## 2a Solution

The above is not a Markov process. However, we can transform it into one by defining the appropriate state space. Let $X_n$, n= 0, 1, 2, ..., represent the weather over the past three days. Then $X_n$ is a Markov chain with the state space $S = \{\text{Sunny today and yesterday, Sunny today but rainy yesterday, Rainy three days in a row, Rainy two days in a row, Rainy today}\}$ and probability transition matrix $\mathbf{P}$ given by

\begin{bmatrix}
  0.5 & 0 & 0 & 0 & 0.5 \\
  0.7 & 0 & 0 & 0 & 0.3 \\
  0 & 0.9 & 0.1 & 0 & 0 \\
  0 & 0.75 & 0.25 & 0 & 0 \\
  0 & 0.6 & 0 & 0.4 & 0
\end{bmatrix}

We can represent this Markov chain by the directed graph depicted below

## 2b Solution


For any $n \in \mathbb{N}$, the $nth$ day probability vector $\mathbf{v}_n$ is given by
$$
\mathbf{v}_n = \mathbf{v}_0 ⋅ \mathbf{P}
$$
Where $\mathbf{v}_0$ is the initial state vector and $\mathbf{P}$ is the transition probability matrix.

In [ ]:

def rainy_probability(n):
  #Purpose: Find the exact probability on day n given that is sunny today and sunny yesterday
  # -------------------
  #Input: 
  #  n = integer number of days in the future
  #Output:
  # A single number which is the probabaility it is rainy on day n
  # -------------------

  
  # Initial state vector
  v0 = jnp.array([1., 0., 0., 0., 0.])

  # Transition matrix
  P = jnp.array([[0.5, 0., 0., 0., 0.5],
                [0.7, 0., 0., 0., 0.3],
                [0., 0.9, 0.1, 0., 0.],
                [0, 0.75, 0.25, 0., 0.],
                [0., 0.6, 0., 0.4, 0.]])

  # Probability vector for day n
  vn = jnp.matmul(v0, jnp.linalg.matrix_power(P, n))
  
  # It is rainy in the last three states so we sum their probabilites to get the probability of rain on day n
  pRain = vn[2:].sum()

  return pRain 

In [ ]:
# E.g., 7-day forecast
print(rainy_probability(7))

0.38536853


## 2c Solution



Every Markov chain has a stationary distribution which satisfies

$$
\mathbf{v}_\infty = \mathbf{v}_\infty \cdot \mathbf{\text{P}} 
$$

where $\mathbf{v}_\infty = \lim_{n \to \infty} \mathbf{v}_n = \lim_{n \to \infty} \mathbf{v}_0 \cdot \mathbf{\text{P}}^n$
\begin{align}
  [p_1, p_2, p_3, p_4, p_5] &= [p_1, p_2, p_3, p_4, p_5] \begin{bmatrix}
  0.5 & 0 & 0 & 0 & 0.5 \\
  0.7 & 0 & 0 & 0 & 0.3 \\
  0 & 0.9 & 0.1 & 0 & 0 \\
  0 & 0.75 & 0.25 & 0 & 0 \\
  0 & 0.6 & 0 & 0.4 & 0
\end{bmatrix}
\end{align}

Thus we seek the eigenvector associated with the eigenvalue of 1.

 
 Thus we have five equations in five unknowns
\begin{align}
0.5p_1 + 0.7p_2 &= p_1 \implies -0.5p_1 + 0.7p_2 &= 0 \qquad (1)\\
0.9p_3 + 0.75p_4 + 0.6p_5 &= p_2 \implies -p_2 + 0.9p_3 + 0.75p_4 + 0.6p_5 & = 0\qquad (2) \\
0.1p_3 + 0.25p_4 &= p_3 \implies -0.9p_3 +0.25p_4 &= 0 \qquad (3) \\
0.4p_5 &= p_4 \implies -p_4 + 0.4p_5 &= 0 \qquad (4)\\
0.5p_1 + 0.3p_2 &= p_5 \implies 0.5p_1 + 0.3p_2 - p_5 &= 0\qquad (5)
\end{align}

And so we have the form $\text{A}\mathbf{x} = \mathbf{b}$ where 
$$\text{A} = \begin{bmatrix}
-0.5 & 0.7 & 0 & 0 & 0\\
0 & -1 & 0.9 & 0.75 & 0.6\\
0 & 0 & -0.9 & 0.25 & 0\\
0 & 0 & 0 & -1 & 0.4\\
0.5 & 0.3 & 0 & 0 & -1\\
\end{bmatrix};
\qquad\mathbf{x}= [p_1, p_2, p_3, p_4, p_5];
\qquad\mathbf{b} = \mathbf{0}
$$

By row reducing we find that there is an infinite number of solutions. However, we require that $p_1 + p_2 + p_3 + p_4 + p_5 = 1$


In [7]:

def limit_rainy_probability():
  #Purpose: Find the exact limiting probability it is rainy in the far future
  #Output: 
  # A single number which is the value of the limit

  a = jnp.array([[-0.5, 0.7, 0., 0., 0.],
               [0., -1., 0.9, 0.75, 0.6],
               [0., 0., -0.9, 0.25, 0.],
               [0., 0., 0., -1., 0.4],
               [0.5, 0.3, 0., 0., -1.]])

  # b = [0., 0., 0., 0., 0.]

  # v_infty = jnp.linalg.solve(a, b)

  w, v = jnp.linalg.eig(a)

  return w, v

In [9]:
print(limit_rainy_probability()[0])

[ 1.4762918e-08+0.j         -1.0107117e+00+0.45007744j
 -1.0107117e+00-0.45007744j -1.1892887e+00+0.15289j
 -1.1892887e+00-0.15289j   ]


#Problem 3 - Monkey at a typewriter

## 3a Solution

## 3b Solution

##3c Solution

In [ ]:
def expected_monkey_time(N_buttons, word_sequence):
  #Purpose:
  #   Returns the expected amount of time for the monkey to type the word
  #Input: 
  #  N_buttons = An integer, number of buttons on the typewritter
  #  word_sequence = An array of shape (N_word,). Each enty is an integer in [0,N_buttons-1]
  #Output:
  #  A real number which is the expected time until the word appears

## 3d Solution

In [ ]:
def expected_mokey_time_tests():
  score = 0
  score += (expected_mokey_time(5,jnp.array[0]) == 5)

# Problem 4 - Generalized PIG


## 4a Solution

## 4b Solution
  

In [ ]:
def generalized_pig_E_and_P(target_score,p_bust,v_advance):
  #Purpose:
  #  Returns the expected value of playing the startegy of re-rolling until you hit target_score in generlized Pig
  #Input:
  #  target_score = Integer
  #  p_bust = real in (0,1), probability to go bust on any given roll
  #  v_advance = an array of shape (N_max,) indicating the probability to advance on a non-bust roll

  E_score = target_score
  P_score = 0.5
  return E_score, P_score

# Problem 5 - Simple Can't Stop Roll Again AI

## 5a Solution

In [ ]:
def cant_stop_bust_probability(runner_col,illegal_col):
  #Purpose:
  #  Compute the bust_probability if we were to roll again in Can't Stop
  #Input:
  #  runner_col = an array of shape (11,) of integers with the runner locations
  #  illegal_col = an array of shape (11,) of boolean with which columns are illegal to play in 
  #NOTE:
  #  We assume N_Max_Runners = 3 for this one!

  return 0.5

## 5b Solution

In [ ]:
def simple_roll_again_AI(runner_col, illegal_col):
  #Purpose:
  #  Determine wheter or not to roll again or not in Can't stop given ONLY the current state of the runners and the illegal columns
  #  (Note: A better AI would take into account the player positions too, but we are making a very simple AI here)
  #Input:
  #  runner_col = an array of shape (11,) of integers with the runner locations
  #  illegal_col = an array of shape (11,) of boolean with which columns are illegal to play in 
  #NOTE:
  #  We assume N_Max_Runners = 3 for this one!

  roll_again = bool(random.randint(0,1))
  return roll_again